In [1]:
import matplotlib.pyplot as plt
from collections import defaultdict
import numpy as np
import seaborn as sns
import os
import pickle
import sys
sys.path.append("..")
from tqdm import tqdm
import networkx as nx
from tools.opera_tools import gen_graphx, gen_x_y_dataset, load_mc
from tools.opera_tools import BRICK_X_MAX, BRICK_X_MIN, BRICK_Y_MAX, BRICK_Y_MIN, SAFE_M

In [2]:
NUM_SHOWERS_IN_BRICK = 200

In [3]:
import os
import psutil
from sklearn.metrics import roc_auc_score, average_precision_score, accuracy_score, log_loss
from sklearn.metrics import precision_recall_curve
from IPython.display import clear_output
import sys
sys.path.append("..")
from sklearn.linear_model import TheilSenRegressor
from copy import deepcopy      
from collections import Counter
EPS = 1e-6

In [4]:
def load_mc(filename="mcdata_taue2.root", step=1):
    f = uproot.open(filename)
    mc = f['Data'].pandas.df(["Event_id", "ele_P", "BT_X", "BT_Y",
                              "BT_Z","BT_SX", "BT_SY","ele_x", 
                              "ele_y", "ele_z", "ele_sx", "ele_sy", "chisquare", ], flatten=False)
    pmc = pd.DataFrame(mc)
    pmc['numtracks'] = pmc.BT_X.apply(lambda x: len(x))
    # cuts
    shapechange = [pmc.shape[0]]
    pmc = pmc[pmc.ele_P > 0.1]
    shapechange.append(pmc.shape[0])

    pmc = pmc[pmc.ele_z < 0]
    shapechange.append(pmc.shape[0])

    pmc = pmc[pmc.numtracks > 3]
    shapechange.append(pmc.shape[0])
    print("numtracks reduction by cuts: ", shapechange)
    pmc['m_BT_X'] = pmc.BT_X.apply(lambda x: x.mean())
    pmc['m_BT_Y'] = pmc.BT_Y.apply(lambda x: x.mean())
    pmc['m_BT_Z'] = pmc.BT_Z.apply(lambda x: x.mean())

    print("len(pmc): {len}".format(len=len(pmc)))
    return pmc

In [5]:
from tools.opera_tools import plot_graphx, DISTANCE, scattering_estimation_loss
from tools.opera_tools import gen_graphx, gen_x_y_dataset, load_mc
pmc = load_mc(filename='./data/mcdata_taue2.root', step=1)

numtracks reduction by cuts:  [18724, 18679, 9616, 9106]
len(pmc): 9106


## Генерация данных

In [6]:
from math import fabs, sqrt, log
def rms_integral_root_closed_py(basetrack_left, basetrack_right, 
                             TX_LEFT='TX', TY_LEFT='TY',
                             TX_RIGHT='TX', TY_RIGHT='TY'):
    """
    Метрика близости между двумя треками
    """
    
    dz = basetrack_right['features']['SZ'] - basetrack_left['features']['SZ']
    dx = basetrack_left['features']['SX'] - (basetrack_right['features']['SX'] - basetrack_right['features']['TX'] * dz)
    dy = basetrack_left['features']['SY'] - (basetrack_right['features']['SY'] - basetrack_right['features']['TY'] * dz)
    dtx = (basetrack_left['features']['TX'] - basetrack_right['features']['TX'])
    dty = (basetrack_left['features']['TY'] - basetrack_right['features']['TY'])
    # dz can be assigned to arbitrary value, acutally !
    a = (dtx * dz) ** 2 + (dty * dz) ** 2
    b = 2 * (dtx * dz * dx +  dty * dz * dy)
    c = dx ** 2 + dy ** 2
    if a == 0.:
        return fabs(sqrt(c))
    discriminant = (b ** 2 - 4 * a * c)
    log_denominator = 2 * sqrt(a) * sqrt(a + b + c) + 2 * a + b + EPS
    log_numerator = 2 * sqrt(a) * sqrt(c) + b + EPS
    first_part = ( (2 * a + b) * sqrt(a + b + c) - b * sqrt(c) ) / (4 * a)
    if fabs(discriminant) < EPS:
        return fabs(first_part)

    result = fabs((discriminant * log(log_numerator / log_denominator) / (8 * sqrt(a * a * a)) + first_part))
    return result

def class_disbalance_graphx(graphx):
    signal = []
    for _, node in graphx.nodes(data=True):
        signal.append(node['signal'])
    return list(zip(*np.unique(signal, return_counts=True)))

def class_disbalance_graphx__(graphx):
    signal = []
    for _, node in graphx.nodes(data=True):
        signal.append(node['signal'])
    return np.unique(signal, return_counts=True)

In [7]:
def scale_data(pmc, scale = 10000):
    """
    Приводим данные к нормальному разрешению так что нейронкам будет приятнее работать.
    """
    showers = []
    for idx in pmc.index:
        shower = pmc.loc[idx]
        
        showers.append(
            {
                'TX': shower['BT_X'] / scale,
                'TY': shower['BT_Y'] / scale,
                'TZ': shower['BT_Z'] / scale,
                'PX': shower['BT_SX'],
                'PY': shower['BT_SY'],
                'PZ': np.ones_like(shower['BT_X']),
                'ele_P': shower['ele_P'],
                'ele_TX': shower['ele_x'] / scale,
                'ele_TY': shower['ele_y'] / scale,
                'ele_TZ': shower['ele_z']  / scale,
                'ele_PX': shower['ele_sx'],
                'ele_PY': shower['ele_sy'],
                'ele_PZ': 1.
            }
        )
    return showers

selected_showers = scale_data(pmc)
selected_showers = [selected_shower for selected_shower in selected_showers if len(selected_shower['PX']) > 70]
selected_showers = [selected_shower for selected_shower in selected_showers if len(selected_shower['PX']) < 3000]

In [8]:
len(selected_showers)

8019

In [9]:
def gen_bricks(selected_showers, NUM_SHOWERS_IN_BRICK=25):
    """
    Функция для генерации кирпичей.
    """
    bricks = []
    scale = 10000
    bricks = []
    
    X_SIZE = ((BRICK_X_MAX - SAFE_M) / scale / 2) / 2 # default ((BRICK_X_MAX - SAFE_M) / scale / 2)
    Y_SIZE = ((BRICK_Y_MAX - SAFE_M) / scale / 2) / 2 # default ((BRICK_Y_MAX - SAFE_M) / scale / 2)
    num_processed_showers = 0
    while num_processed_showers < len(selected_showers):
        to_process_showers = np.random.randint(NUM_SHOWERS_IN_BRICK - 5, NUM_SHOWERS_IN_BRICK + 5)
        node_id = 0
        nodes_to_add = []
        showers_data = []
        for j in range(to_process_showers):
            num_processed_showers += 1
            if num_processed_showers >= len(selected_showers): break
            selected_shower = selected_showers[num_processed_showers]
            x_diff = np.random.uniform(low=-X_SIZE + 1, high=X_SIZE - 1) - np.median(selected_shower['TX'])
            y_diff = np.random.uniform(low=-Y_SIZE + 1, high=Y_SIZE - 1) - np.median(selected_shower['TY'])
            showers_data.append(
                {
                'numtracks': len(selected_shower['PX']),
                'signal': j,
                'ele_P': selected_shower['ele_P'],
                'ele_SX': (selected_shower['ele_TX'] + x_diff) * scale,
                'ele_SY': (selected_shower['ele_TY'] + y_diff) * scale,
                'ele_SZ': selected_shower['ele_TZ'] * scale,
                'ele_TX': selected_shower['ele_PX'] / selected_shower['ele_PZ'],
                'ele_TY': selected_shower['ele_PY'] / selected_shower['ele_PZ']
                }
            )
            for k in range(len(selected_shower['PX'])):
                nodes_to_add.append(
                    (
                        node_id,
                        {
                            'features': {
                                'SX': (selected_shower['TX'][k] + x_diff) * scale,
                                'SY': (selected_shower['TY'][k] + y_diff) * scale,
                                'SZ': selected_shower['TZ'][k] * scale,
                                'TX': selected_shower['PX'][k] / selected_shower['PZ'][k],
                                'TY': selected_shower['PY'][k] / selected_shower['PZ'][k],
                            },
                            'signal': j
                        }
                    )
                )
                node_id += 1
        graphx = nx.DiGraph()
        graphx.add_nodes_from(nodes_to_add)
        graphx.graph['showers_data'] = showers_data
        bricks.append(graphx)
    return bricks

In [10]:
bricks = gen_bricks(selected_showers=selected_showers, NUM_SHOWERS_IN_BRICK=25)

In [11]:
len(bricks)

329

In [12]:
import pandas as pd
from tqdm import tqdm_notebook

def digraph_to_csv(graphs: list):
    df = pd.DataFrame(columns=['brick_id', 'shower_id', 'SX', 'SY', 'SZ', 'TX', 'TY'])
    for i, graph in tqdm_notebook(enumerate(graphs)):
        nodes = graph.nodes()
        SX = [node['features']['SX'] for node in nodes.values()]
        SY = [node['features']['SY'] for node in nodes.values()]
        SZ = [node['features']['SZ'] for node in nodes.values()]
        TX = [node['features']['TX'] for node in nodes.values()]
        TY = [node['features']['TY'] for node in nodes.values()]
        shower_id = [node['signal'] for node in nodes.values()]
        brick_id = [i for _ in range(len(shower_id))]
        df = df.append(
            pd.DataFrame(
                {'brick_id': brick_id, 
                 'shower_id': shower_id, 
                 'SX': SX, 
                 'SY': SY, 
                 'SZ': SZ, 
                 'TX': TX, 
                 'TY': TY
                }
            )
        )
    return df
        

def csv_to_digraph(df: pd.DataFrame):
    bricks = []
    for name, group in df.groupby('brick_id'):
        print(group.shape)
        nodes_to_add = []

        for node_id, row in group.iterrows():
            nodes_to_add.append(
                (
                    node_id,
                    {
                        'features': {
                            'SX': row.SX,
                            'SY': row.SY,
                            'SZ': row.SZ,
                            'TX': row.TX,
                            'TY': row.TY,
                        },
                        'signal': row['shower_id']
                    }
                )
            )
        graphx = nx.DiGraph()
        graphx.add_nodes_from(nodes_to_add)
        bricks.append(graphx)
    return bricks

In [13]:
df = digraph_to_csv(bricks)

In [14]:
len(df)

4322387

In [15]:
from sklearn.model_selection import GroupShuffleSplit
gsplit = GroupShuffleSplit(n_splits=2, test_size=0.3)
splits = gsplit.split(df, groups=df.brick_id)
df_train, df_test = list(splits)[0]

df_train = df.iloc[df_train]
df_test = df.iloc[df_test]

In [16]:
len(df_train)

3013026

In [17]:
len(df_test)

1309361

In [18]:
len(df_train.brick_id.unique())

230

In [19]:
len(df_test.brick_id.unique())

99

In [23]:
from sklearn.metrics import fowlkes_mallows_score
def scorer(labels_true, labels_pred, groups=None):
    if groups is None:
        return fowlkes_mallows_score(labels_true=labels_true, labels_pred=labels_pred)
    fowlkes_mallows = 0.
    for group in np.unique(groups):
        fowlkes_mallows += fowlkes_mallows_score(labels_true=labels_true[groups==group], 
                                                 labels_pred=labels_pred[groups==group])
    return fowlkes_mallows / len(np.unique(groups))



In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GroupShuffleSplit
# df = pd.read_csv('./opera_all_data.csv')

gsplit = GroupShuffleSplit(n_splits=3, test_size=0.3)
splits = gsplit.split(df, groups=df.brick_id)
train, test = list(splits)[0]

df_train = df.iloc[train]
df_test = df.iloc[test]


from sklearn.model_selection import GroupShuffleSplit
gsplit = GroupShuffleSplit(n_splits=3, test_size=0.5)
splits = gsplit.split(df_test, groups=df_test.brick_id)
val, test = list(splits)[0]

df_val = df.iloc[val]
df_test = df.iloc[test]

In [ ]:
np.savetxt('opera_train.data', df_train[['brick_id', 'SX', 'SY', 'SZ', 'TX', 'TY']].values, 
           fmt=['%i', '%5.2f', '%5.2f', '%5.2f', '%1.4f', '%1.4f'])
np.savetxt('opera_train.solution', df_train[['brick_id', 'shower_id']].values.astype(int), fmt='%i')

In [ ]:
np.savetxt('opera_valid.data', df_val[['brick_id', 'SX', 'SY', 'SZ', 'TX', 'TY']].values, 
           fmt=['%i', '%5.2f', '%5.2f', '%5.2f', '%1.4f', '%1.4f'])
np.savetxt('opera_valid.solution', df_val[['brick_id', 'shower_id']].values.astype(int), fmt='%i')

In [ ]:
np.savetxt('opera_test.data', df_test[['brick_id', 'SX', 'SY', 'SZ', 'TX', 'TY']].values, 
           fmt=['%i', '%5.2f', '%5.2f', '%5.2f', '%1.4f', '%1.4f'])
np.savetxt('opera_test.solution', df_test[['brick_id', 'shower_id']].values.astype(int), fmt='%i')